## Ejercicio - Regresión Viviendas Barcelona

- Utilizando el dataset de publicaciones de venta de viviendas en Barcelona _**viviendas_barcelona.parquet**_, entrena un modelo de regresión lineal múltiple que sea capaz de calcular el precio de venta:

    - Antes de crear el modelo y entrenarlo, haz un **EDA** de los datos.
        - ¿Qué columnas pueden ser las más importantes?
        - ¿Como se distribuyen los precios?
        - ¿Qué columnas sería factible limpiar?
        - ¿Qué significan los **np.nan**?
     
    - Cuando entiendas bien los datos con los que estás tratando, puedes proceder a realizar un preprocesamiento:
        - Elimina duplicados.
        - Elimina columnas innecesarias
        - Trata los valores perdidos **np.nan**.
        - Maneja los outliers como consideres.
        - Realiza transformaciones a los datos que consideres necesarias.
        - Codifica los datos categóricos para que el modelo los entienda (pásalos a números).
        - Considera normalizar los datos antes de utilizarlos para el entreno.
        
    - Entrena el modelo con un subconjunto de 70-80% del dataset.
      
    - Utiliza los datos restantes para comprobar cómo de bueno es el modelo:
        - Calcula **R2**, **MAE** y **MSE**.
        
- Importa las librerías que vayas a necesitar.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Modelo
from sklearn.linear_model import LinearRegression

# Herramientas de preprocesamiento
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, TargetEncoder, OneHotEncoder

# Split
from sklearn.model_selection import train_test_split

# Métricas
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

df = pd.read_parquet("../Data/viviendas_barcelona.parquet")

In [2]:
df.isna().sum()

price                      510
title                      350
province                     0
location                   606
lat                        606
lng                        606
agency                     350
updated                    350
timestamp                    0
id_1                         0
id_2                         0
Superficie construida      501
Emisiones                11144
Clasificación              350
Baños                     1294
Consumo                  25347
Habitaciones              1308
Referencia                 350
Antigüedad               18755
Garaje                   18945
Terraza                  14711
Se aceptan mascotas      25968
Ascensor                 15621
Tipo de casa             25845
Superficie útil           8745
dtype: int64

* EDA